In [28]:
import pandas as pd
import numpy as np

In [23]:
x = set((3, 4, 5))
x - {3, 5}

{4}

In [24]:
outfit = pd.read_csv("../data/outfit.csv")[['outfit_id', 'gender', 'tags']]
outfit['tags'] = outfit['tags'].apply(lambda x: set(x.lstrip("{").rstrip("}").split(",")) - {'남성', '여성'})
outfit2idx = {v:i for i,v in enumerate(outfit['outfit_id'].unique())}
outfit['outfit_id'] = outfit['outfit_id'].map(outfit2idx)
outfit

,outfit_id,gender,tags
0,0,F,"{반팔 티셔츠, 스트릿, 가을}"
1,1,F,"{봄, 스니커즈, 캐주얼, 나이키}"
2,2,F,"{여름, 셔츠, 워커, 레더 스커트, 캐주얼}"
3,3,F,"{숄더 백, 여름, 스니커즈, 나이키, 트랙 팬츠, 슬리브리스, 로맨틱}"
4,4,M,"{스트릿, 여름, 스니커즈, 나이키, 데님 팬츠, 크로스 백, 베스트, 비니}"
...,...,...,...
10424,10424,M,"{스트릿, 여름}"
10425,10425,F,"{걸리시, 여름}"
10426,10426,F,"{스트릿, 여름}"
10427,10427,F,"{스트릿, 여름}"


In [35]:
male = outfit[outfit['gender']=='M'].reset_index(drop=True).copy()
male

,outfit_id,gender,tags
0,4,M,"{스트릿, 여름, 스니커즈, 나이키, 데님 팬츠, 크로스 백, 베스트, 비니}"
1,6,M,"{스트릿, 여름, 스니커즈, 반팔 셔츠, 트랙 팬츠}"
2,7,M,"{스트릿, 여름, 스니커즈, 데님 팬츠, 크로스 백, 비니, 반팔 티셔츠}"
3,8,M,"{여름, 스니커즈, 타미힐피거, 슬랙스, 비니, 반팔 티셔츠, 캐주얼}"
4,10,M,"{스트릿, 여름, 크로스 백, 닥터마틴, 점프 수트, 워커, 반팔 티셔츠, ㄴ마성}"
...,...,...,...
3043,10399,M,"{카라셔츠, 여름, 캐주얼, 드로우핏}"
3044,10406,M,"{여름, 캐주얼}"
3045,10410,M,"{스트릿, 여름}"
3046,10414,M,"{댄디, 여름}"


In [36]:
female = outfit[outfit['gender']=='F'].reset_index(drop=True).copy()
female

,outfit_id,gender,tags
0,0,F,"{반팔 티셔츠, 스트릿, 가을}"
1,1,F,"{봄, 스니커즈, 캐주얼, 나이키}"
2,2,F,"{여름, 셔츠, 워커, 레더 스커트, 캐주얼}"
3,3,F,"{숄더 백, 여름, 스니커즈, 나이키, 트랙 팬츠, 슬리브리스, 로맨틱}"
4,5,F,"{숄더 백, 걸리시, 여름, 스커트, 피케이, 샌들, 아디다스, 반팔 티셔츠}"
...,...,...,...
7376,10423,F,"{스트릿, 여름}"
7377,10425,F,"{걸리시, 여름}"
7378,10426,F,"{스트릿, 여름}"
7379,10427,F,"{스트릿, 여름}"


In [37]:
def jaccard_similarity(tags1: set, tags2: set):
    intersection = tags1.intersection(tags2)
    union = tags1.union(tags2)
    similarity = len(intersection) / len(union)
    return similarity

In [91]:
def get_k_similar(id: int, k: int, th: float, tags: pd.Series):
    similarity = list()
    for other in range(len(tags)):
        if id == other:
            similarity.append(0)
        else:
            similarity.append(jaccard_similarity(tags[id], tags[other]))
    similarity = np.array(similarity)
    # k_similar = np.argsort(similarity)[-k:]
    k_similar = np.where(similarity > th)[0]
    return k_similar

In [92]:
get_k_similar(1, 5, 0.8, female['tags'])

array([1015, 1128, 1155, 1165, 1347, 1591, 2002, 2055, 2104, 2138, 2193,
       2217, 2231], dtype=int64)

In [93]:
female.iloc[[1, 1015, 1128, 1155, 1165, 1347, 1591, 2002, 2055, 2104, 2138, 2193,
        2217, 2231],:]

,outfit_id,gender,tags
1,1,F,"{봄, 스니커즈, 캐주얼, 나이키}"
1015,1732,F,"{봄, 스니커즈, 캐주얼, 나이키}"
1128,1896,F,"{봄, 스니커즈, 캐주얼, 나이키}"
1155,1942,F,"{봄, 스니커즈, 캐주얼, 나이키}"
1165,1953,F,"{봄, 스니커즈, 캐주얼, 나이키}"
1347,2259,F,"{봄, 스니커즈, 캐주얼, 나이키}"
1591,2619,F,"{봄, 스니커즈, 캐주얼, 나이키}"
2002,3173,F,"{봄, 스니커즈, 캐주얼, 나이키}"
2055,3252,F,"{봄, 스니커즈, 캐주얼, 나이키}"
2104,3318,F,"{봄, 스니커즈, 캐주얼, 나이키}"


In [94]:
m_similars = list()
for id in range(len(male)):
    m_similars.append(get_k_similar(id, 5, 0.8, male['tags']))
    
print(len(m_similars))

3048


In [100]:
m_similars2 = list()
for id in range(len(male)):
    m_similars2.append(get_k_similar(id, 5, 0.5, male['tags']))
    
print(len(m_similars2))

3048


In [95]:
m_similars[0]

array([40], dtype=int64)

In [101]:
from collections import deque
from collections import defaultdict

def bfs(similars, start, visited, cluster, cluster_id):
    q = deque([start])
    while q:
        now = q.popleft()
        for x in similars[now]:
            if not visited[x]:
                q.append(x)
                visited[x] = True
                cluster[cluster_id].append(x)
                if len(cluster[cluster_id]) == 300:
                    return
                
cluster = defaultdict(list)
visited = [False] * len(m_similars)
for i in range(len(m_similars)):
    if not visited[i]:
        cluster_id = len(cluster) + 1
        bfs(m_similars2, i, visited, cluster, cluster_id)

In [102]:
cluster.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163])

In [103]:
for _, c in cluster.items():
    print(_, len(c))

1 300
2 5
3 300
4 2
5 2
6 7
7 4
8 3
9 6
10 2
11 69
12 2
13 18
14 2
15 16
16 3
17 2
18 3
19 5
20 7
21 2
22 300
23 48
24 300
25 2
26 2
27 4
28 2
29 2
30 10
31 3
32 3
33 300
34 46
35 8
36 29
37 13
38 56
39 2
40 8
41 4
42 3
43 2
44 9
45 12
46 3
47 67
48 4
49 3
50 4
51 4
52 6
53 2
54 2
55 2
56 2
57 3
58 2
59 4
60 4
61 2
62 3
63 5
64 5
65 2
66 2
67 3
68 7
69 3
70 5
71 2
72 3
73 2
74 2
75 3
76 2
77 2
78 5
79 6
80 2
81 4
82 3
83 2
84 2
85 3
86 2
87 2
88 4
89 2
90 2
91 2
92 2
93 2
94 5
95 6
96 2
97 2
98 2
99 2
100 2
101 2
102 3
103 3
104 2
105 2
106 98
107 2
108 2
109 2
110 6
111 2
112 8
113 4
114 19
115 3
116 2
117 2
118 4
119 2
120 3
121 3
122 8
123 2
124 6
125 2
126 2
127 2
128 2
129 2
130 3
131 2
132 2
133 2
134 8
135 3
136 2
137 3
138 4
139 5
140 2
141 4
142 2
143 2
144 2
145 13
146 26
147 10
148 6
149 6
150 19
151 2
152 2
153 2
154 3
155 2
156 3
157 7
158 13
159 4
160 2
161 5
162 2
163 2


In [56]:
cluster[3]

[294]